In [1]:
# En vez de recoger la TF&C, recoge la venta neta, ya que la TF&C puede estar abierta por canal

import sys
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
mode = 'PC'
paises = ['MX'] #['CL', 'MX', 'CO', 'AR', 'PE', 'UY','BR']

#nombre_sheets_ppto = 'Copy of Presupuesto Retail Digital 2024'
nombres_sheets_ppto = ['Presupuesto Retail Digital 2024', 'Presupuesto Retail Digital 2025']

#nombre_sheets_ppto = 'Presupuesto Retail Digital 2024 prueba MX 240116'

In [3]:
carpeta_modulos = '../../Modulos/'  #/home/data/python/performance_automation/Modulos
carpeta_modulos_propios = '../Modulos/' 
carpeta_resultados = '../Resultados/'
carpeta_input = '../Inputs/'

In [4]:
if mode == 'VM': # Esto tiene que ir en funcion generar dag
    carpeta_modulos = '/home/data/python/performance_automation/Modulos/'

sys.path.insert(0, carpeta_modulos)
import funciones_transversales as tf

### Metas resumen

In [5]:
simbolos = [',', '%', '$', '-', ' ']

In [6]:
metricas = {
            '2024': {'VENTA_NETA': [4, 6], 'VENTA_COLOCADA': [41, 43], 'ORDENES': [46, 48], 'VISITAS': [56, 58], 'INVERSION': [62, 62], 'IMPUESTO': [39, 39], 'PORCENTAJE_TRAFICO_ORGANICO': [60, 60]},
            '2025': {'VENTA_NETA': [4, 6], 'VENTA_COLOCADA': [45, 47], 'ORDENES': [50, 52], 'VISITAS': [60, 62], 'INVERSION': [66, 66], 'IMPUESTO': [42, 42], 'PORCENTAJE_TRAFICO_ORGANICO': [64, 64]}
            }

#, 'TFYC': [38, 38]


#  declaracion explícita de las filas (se ajustan solas) [incluidos desde y hasta]
#  Ordenes son colocadas

In [7]:

df_resumen_metas_general_all = pd.DataFrame()
df_resumen_metas_otros_all = pd.DataFrame()

for nombre_sheets_ppto in nombres_sheets_ppto:
    print(nombre_sheets_ppto)
    año_planificacion = nombre_sheets_ppto.split(' ')[-1]
    #if año_planificacion == "2024":
    #    continue
    df_resumen_metas = pd.DataFrame()
    for pais in paises:
        print(pais)
        df = tf.lectura_segura_Gsheets(nombre_sheets_ppto, pais)
        
        df_pais = pd.DataFrame()
        for metrica in metricas[año_planificacion]:
            print('     ', metrica)
            [i_0, i_f] = metricas[año_planificacion][metrica]
            i_0, i_f = i_0 - 2, i_f - 1
            
            #print(metrica)
            
            #print('DF0')
            #display(df.iloc[i_0:i_f])
            df_metrica = df.iloc[i_0:i_f, 1:14].reset_index(drop = True)
            
            #display(df_metrica)
            
            df_metrica.columns = ['CANAL'] + [m for m in range(1, 13)]
            df_metrica['CANAL'] = np.where(df_metrica['CANAL'] == 'APP SODIMAC', 'APP', np.where(df_metrica['CANAL'] == 'WEB SODIMAC', 'WEB', np.where(df_metrica['CANAL'] == 'FCOM / LINIO', 'FCOM', np.where(df_metrica['CANAL'] == 'FCOM', 'FCOM', ''))))
            
            if pais not in ['CL', 'CO', 'PE']: # Para paises no andes, solo se considera app y web
                df_metrica = df_metrica[df_metrica['CANAL'].isin(['WEB', 'APP', ''])].reset_index(drop = True)
            
            if pais == 'CO': # Por ahora, ya que las métricas no están completas   # CASO CO
                df_metrica = df_metrica[df_metrica['CANAL'].isin(['WEB', 'APP', ''])].reset_index(drop = True)
            
            #display(df_metrica)

            for m in range(1, 13):

                for s in simbolos:
                    df_metrica[m] = df_metrica[m].str.replace(s, '')
                df_metrica[m] = df_metrica[m].replace('', 0).astype(float)
                df_metrica[m] = df_metrica[m].fillna(0)
                
                if metrica in ['TFYC', 'IMPUESTO']:
                    df_metrica[m] = df_metrica[m] / 100
            
            #display(df_metrica)
            
            df_metrica.insert(0, 'METRICA', metrica)
            df_metrica.insert(0, 'PAIS', pais)
            
            #sys.exit()
            
            df_pais = pd.concat([df_pais, df_metrica], axis = 0, ignore_index = True)
        
        df_resumen_metas = pd.concat([df_resumen_metas, df_pais], axis = 0, ignore_index = True)

    df_resumen_metas_base = df_resumen_metas[df_resumen_metas['CANAL'] == '']

    for canal in df_resumen_metas['CANAL'].unique():
        if canal == '':
            continue
        #print(canal)
        df_resumen_metas_canal = df_resumen_metas[df_resumen_metas['CANAL'].isin([canal, ""])]
        
        #display(df_resumen_metas_canal)
        
        df_resumen_metas_canal = df_resumen_metas_canal.drop(columns = 'CANAL')
        df_resumen_metas_canal = pd.melt(df_resumen_metas_canal, id_vars = ['PAIS', 'METRICA'], var_name = 'MES', value_name = 'VALOR')
        df_resumen_metas_canal = df_resumen_metas_canal.pivot_table(index = ['PAIS', 'MES'], columns = 'METRICA', values = 'VALOR').reset_index()
        df_resumen_metas_canal['TFYC'] = 1 - (df_resumen_metas_canal['VENTA_NETA'] / df_resumen_metas_canal['VENTA_COLOCADA']) * (1 + df_resumen_metas_canal['IMPUESTO'])
        #df_resumen_metas_canal['ORDENES'] = df_resumen_metas_canal['ORDENES'] / (1 - df_resumen_metas_canal['TFYC'])  # se leen ordenes netas por confiabilidad, pero se llevan a colocadas según relalción
        df_resumen_metas_canal = df_resumen_metas_canal.drop(columns = 'TFYC')
        
        # recupera la estructura inicial con un pivot-melt
        df_resumen_metas_canal = pd.melt(df_resumen_metas_canal, id_vars = ['PAIS', 'MES'], var_name = 'METRICA', value_name = 'VALOR')
        df_resumen_metas_canal = df_resumen_metas_canal.pivot_table(index = ['PAIS', 'METRICA'], columns = 'MES', values = 'VALOR').reset_index()
        df_resumen_metas_canal = df_resumen_metas_canal[df_resumen_metas_canal['METRICA'].isin(['VENTA_NETA', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS'])]
        df_resumen_metas_canal['CANAL'] = canal
        df_resumen_metas_base = pd.concat([df_resumen_metas_base, df_resumen_metas_canal], axis = 0, ignore_index = True)
        
    df_resumen_metas = df_resumen_metas_base.copy()
    df_resumen_metas_general = df_resumen_metas[df_resumen_metas['METRICA'].isin(['VENTA_NETA', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS'])]
    df_resumen_metas_otros = df_resumen_metas[df_resumen_metas['METRICA'].isin(['INVERSION', 'IMPUESTO', 'PORCENTAJE_TRAFICO_ORGANICO'])] # version TFYC SEPARADAS(240209) Se elimna TFYC

    df_resumen_metas_otros = df_resumen_metas_otros.drop(columns = 'CANAL')
    df_resumen_metas_otros = pd.melt(df_resumen_metas_otros, id_vars = ['PAIS', 'METRICA'], var_name = 'MES', value_name = 'VALOR')
    df_resumen_metas_otros = df_resumen_metas_otros.pivot_table(index = ['PAIS', 'MES'], columns = 'METRICA', values = 'VALOR').reset_index()
    df_resumen_metas_otros['PERIODO'] = año_planificacion + '-' + df_resumen_metas_otros['MES'].apply(lambda x: str(x).zfill(2))
    df_resumen_metas_otros_all = pd.concat([df_resumen_metas_otros_all, df_resumen_metas_otros])
    
    # Pivotear y meltear + eliminar ceros
    # Las columnas 1..12 son los meses.  Crear el campo mes (meltear)
    df_resumen_metas_general = pd.melt(df_resumen_metas_general, id_vars = ['PAIS', 'METRICA', 'CANAL'], var_name = 'MES', value_name = 'VALOR')
    # crea PERIODO...mes siempre con dos digitos
    df_resumen_metas_general['PERIODO'] = año_planificacion + '-' + df_resumen_metas_general['MES'].apply(lambda x: str(x).zfill(2))
    display(df_resumen_metas_general)
    
    df_resumen_metas_general_all = pd.concat([df_resumen_metas_general_all, df_resumen_metas_general], axis = 0, ignore_index = True)

df_resumen_metas_otros = df_resumen_metas_otros_all.copy().reset_index(drop = True)
df_resumen_metas_general = df_resumen_metas_general_all.copy().reset_index(drop = True)
df_resumen_metas_general

Presupuesto Retail Digital 2024
MX
      VENTA_NETA
      VENTA_COLOCADA
      ORDENES
      VISITAS
      INVERSION
      IMPUESTO
      PORCENTAJE_TRAFICO_ORGANICO


,PAIS,METRICA,CANAL,MES,VALOR,PERIODO
0,MX,ORDENES,APP,1,1361.0,2024-01
1,MX,VENTA_COLOCADA,APP,1,6924998.0,2024-01
2,MX,VENTA_NETA,APP,1,4477369.0,2024-01
3,MX,VISITAS,APP,1,132624.0,2024-01
4,MX,ORDENES,WEB,1,3501.0,2024-01
...,...,...,...,...,...,...
91,MX,VISITAS,APP,12,150690.0,2024-12
92,MX,ORDENES,WEB,12,3848.0,2024-12
93,MX,VENTA_COLOCADA,WEB,12,16993505.0,2024-12
94,MX,VENTA_NETA,WEB,12,10987180.0,2024-12


Presupuesto Retail Digital 2025
MX
      VENTA_NETA
      VENTA_COLOCADA
      ORDENES
      VISITAS
      INVERSION
      IMPUESTO
      PORCENTAJE_TRAFICO_ORGANICO


,PAIS,METRICA,CANAL,MES,VALOR,PERIODO
0,MX,ORDENES,APP,1,1626.0,2025-01
1,MX,VENTA_COLOCADA,APP,1,6977170.0,2025-01
2,MX,VENTA_NETA,APP,1,4571249.0,2025-01
3,MX,VISITAS,APP,1,133189.0,2025-01
4,MX,ORDENES,WEB,1,4099.0,2025-01
...,...,...,...,...,...,...
91,MX,VISITAS,APP,12,216078.0,2025-12
92,MX,ORDENES,WEB,12,4135.0,2025-12
93,MX,VENTA_COLOCADA,WEB,12,17420680.0,2025-12
94,MX,VENTA_NETA,WEB,12,11413549.0,2025-12


,PAIS,METRICA,CANAL,MES,VALOR,PERIODO
0,MX,ORDENES,APP,1,1361.0,2024-01
1,MX,VENTA_COLOCADA,APP,1,6924998.0,2024-01
2,MX,VENTA_NETA,APP,1,4477369.0,2024-01
3,MX,VISITAS,APP,1,132624.0,2024-01
4,MX,ORDENES,WEB,1,3501.0,2024-01
...,...,...,...,...,...,...
187,MX,VISITAS,APP,12,216078.0,2025-12
188,MX,ORDENES,WEB,12,4135.0,2025-12
189,MX,VENTA_COLOCADA,WEB,12,17420680.0,2025-12
190,MX,VENTA_NETA,WEB,12,11413549.0,2025-12


In [8]:
# pivotear el campo METRICA
df_resumen_metas_general = df_resumen_metas_general.pivot_table(index = ['PAIS', 'CANAL', 'PERIODO'], columns = 'METRICA', values = 'VALOR').reset_index()
df_resumen_metas_general = df_resumen_metas_general[df_resumen_metas_general['VENTA_COLOCADA'] > 0].reset_index(drop = True)
df_resumen_metas_general = df_resumen_metas_general[['PAIS', 'CANAL', 'PERIODO', 'VENTA_NETA', 'VENTA_COLOCADA', 'ORDENES', 'VISITAS']]

In [9]:
dir_metas = '../Data/metas/'
import os

In [10]:
df_resumen_metas_general_all = pd.DataFrame()
if '0_df_resumen_metas_general.parquet' in os.listdir(dir_metas):
    df_resumen_metas_general_all = tf.parquet_act(f'{dir_metas}0_df_resumen_metas_general')

df_resumen_metas_general_all = pd.concat([df_resumen_metas_general, df_resumen_metas_general_all])
df_resumen_metas_general_all = df_resumen_metas_general_all.drop_duplicates(subset = ['PAIS', 'CANAL', 'PERIODO']).reset_index(drop = True)
df_resumen_metas_general_all = df_resumen_metas_general_all.sort_values(['PAIS', 'CANAL', 'PERIODO']).reset_index(drop = True)
tf.parquet_act(f'{dir_metas}0_df_resumen_metas_general', df_resumen_metas_general_all, 'save')

df_resumen_metas_otros_all = pd.DataFrame()
if '1_df_resumen_metas_otros.parquet' in os.listdir(dir_metas):
    df_resumen_metas_otros_all = tf.parquet_act(f'{dir_metas}1_df_resumen_metas_otros')
    
df_resumen_metas_otros_all = pd.concat([df_resumen_metas_otros, df_resumen_metas_otros_all])
df_resumen_metas_otros_all = df_resumen_metas_otros_all.drop_duplicates(subset = ['PAIS', 'PERIODO']).reset_index(drop = True)
df_resumen_metas_otros_all = df_resumen_metas_otros_all.sort_values(['PAIS', 'PERIODO']).reset_index(drop = True)
tf.parquet_act(f'{dir_metas}1_df_resumen_metas_otros', df_resumen_metas_otros, 'save')

### Detalle de venta neta por F-SF

In [11]:
simbolos

[',', '%', '$', '-', ' ']

Solo venta neta por fam-subfam para APP, WEB y FCOM

In [12]:
df_resumen_metas_detalle = pd.DataFrame() # Lectura de detalle por sf

for nombre_sheets_ppto in nombres_sheets_ppto:
    print(nombre_sheets_ppto)
    año_planificacion = nombre_sheets_ppto.split(' ')[-1]
    for pais in paises:

        print(pais)
        
        df = tf.lectura_segura_Gsheets(nombre_sheets_ppto, f'{pais} FAMILIA')

        df = df.iloc[33:240, :15]

        df.columns = ['FAMILIA', 'SUBFAMILIA', 'F-SF'] + [m for m in range(1, 13)]
        df = df[['FAMILIA', 'SUBFAMILIA'] + [m for m in range(1, 13)]]
        
        #display(df.tail())
        
        
        df['FAMILIA'] = np.where(df['FAMILIA'] == '28 - Puertas/Ventanas/Molduras', '02 - Puertas/Ventanas/Molduras', df['FAMILIA'])

        #display(df)
        for m in range(1, 13):
            for s in simbolos:
                df[m] = df[m].str.replace(s, '')
            df[m] = df[m].replace('', 0).astype(float)
            df[m] = df[m].fillna(0)

        df = pd.melt(df, id_vars = ['FAMILIA', 'SUBFAMILIA'], var_name = 'MES', value_name = 'VENTA_NETA')
        df = df[df['VENTA_NETA'] > 0].reset_index(drop = True)

        df['PERIODO'] = año_planificacion + '-' + df['MES'].apply(lambda x: str(x).zfill(2))
        df_agr = df[['PERIODO', 'VENTA_NETA']].groupby(['PERIODO'], as_index = False).sum().rename(columns = {'VENTA_NETA': 'VENTA_NETA_ALL_CHANNELS'})

        df_resumen_metas_general_venta_neta = df_resumen_metas_general[df_resumen_metas_general['PAIS'] == pais]
        df_resumen_metas_general_venta_neta = df_resumen_metas_general_venta_neta[['PERIODO', 'VENTA_NETA']].groupby('PERIODO', as_index = False).sum()
        df_resumen_metas_general_venta_neta = df_resumen_metas_general_venta_neta.merge(df_agr, on = 'PERIODO', how = 'left')
        #display(df_resumen_metas_general_venta_neta)
        df_resumen_metas_general_venta_neta['FACTOR'] = df_resumen_metas_general_venta_neta['VENTA_NETA'] / df_resumen_metas_general_venta_neta['VENTA_NETA_ALL_CHANNELS'] # Solo venta neta por fam-subfam para APP, WEB y FCOM
        df_resumen_metas_general_venta_neta = df_resumen_metas_general_venta_neta[['PERIODO', 'FACTOR']]
        
        #display(df_resumen_metas_general_venta_neta)

        df = df.merge(df_resumen_metas_general_venta_neta, on = 'PERIODO', how = 'left')
        df['VENTA_NETA'] *= df['FACTOR']
        df = df.drop(columns = 'FACTOR')
        df.insert(0, 'PAIS', pais)
        #df['PERIODO'] = año_planificacion + '-' + df['MES'].apply(lambda x: str(x).zfill(2))
        
        df_resumen_metas_detalle = pd.concat([df_resumen_metas_detalle, df])


Presupuesto Retail Digital 2024
MX
Presupuesto Retail Digital 2025
MX


In [13]:
df_resumen_metas_detalle

,PAIS,FAMILIA,SUBFAMILIA,MES,VENTA_NETA,PERIODO
0,MX,01 - Madera Y Tableros,01 - Madera Aserrada,1,90443.253029,2024-01
1,MX,01 - Madera Y Tableros,02 - Tableros,1,158587.892391,2024-01
2,MX,01 - Madera Y Tableros,03 - Tableros Construccion,1,62586.792477,2024-01
3,MX,03 - Fierro/Hierro/Acero,01 - Fierro/Hierro/Acero,1,42233.352147,2024-01
4,MX,03 - Fierro/Hierro/Acero,02 - Perfiles Hierro,1,12887.686763,2024-01
...,...,...,...,...,...,...
2101,MX,02 - Puertas/Ventanas/Molduras,04 - Molduras,12,13112.874706,2025-12
2102,MX,02 - Puertas/Ventanas/Molduras,05 - Escaleras Y Entretechos,12,204.901735,2025-12
2103,MX,02 - Puertas/Ventanas/Molduras,06 - Bloques De Vidrio Y Vidrio,12,9514.967506,2025-12
2104,MX,02 - Puertas/Ventanas/Molduras,07 - Toldos Y Aleros,12,230114.684396,2025-12


In [14]:
df_resumen_metas_detalle_all = pd.DataFrame()
if '2_df_resumen_metas_detalle.parquet' in os.listdir(dir_metas):
    df_resumen_metas_detalle_all = tf.parquet_act(f'{dir_metas}2_df_resumen_metas_detalle')

df_resumen_metas_detalle_all = pd.concat([df_resumen_metas_detalle, df_resumen_metas_detalle_all])
df_resumen_metas_detalle_all = df_resumen_metas_detalle_all.drop_duplicates(subset = ['PAIS', 'PERIODO', 'FAMILIA', 'SUBFAMILIA']).reset_index(drop = True)
df_resumen_metas_detalle_all = df_resumen_metas_detalle_all.sort_values(['PAIS', 'PERIODO', 'FAMILIA', 'SUBFAMILIA']).reset_index(drop = True)
tf.parquet_act(f'{dir_metas}2_df_resumen_metas_detalle', df_resumen_metas_detalle_all, 'save')

In [15]:
print('Proceso Completo')

Proceso Completo
